In [26]:
import numpy as np
from pandas import read_csv
from numpy import cos , sin , sqrt
from scipy.special import lpmv
from scipy.special import factorial as fact

In [27]:
ifn = 'temp_python_input.txt'

In [28]:
data_pd_dataframe = read_csv(ifn,
        skiprows = 0,
        header = None,
        sep='\s+')
matrix_row_file_name = np.asarray(data_pd_dataframe.iloc[:,0])
no_points = np.asarray(data_pd_dataframe.iloc[:,2],dtype=float)
slope = np.asarray(data_pd_dataframe.iloc[:,1],dtype=float)

In [29]:
max_deg = 4

In [30]:
tmp_data = np.asarray( read_csv( matrix_row_file_name[0],
    header = None,
    sep = '\s+',
    dtype = float ))
matrix = tmp_data[:,2]
n = tmp_data[:,0]
m = tmp_data[:,1]

size = np.size(matrix_row_file_name)
for i in np.linspace(1, size -1, size-1, dtype = int):
    row = np.asarray( read_csv( matrix_row_file_name[i],
        header = None,
        sep = '\s+',
        dtype = float ))[:,2]
    matrix = np.column_stack((matrix,row))


In [31]:
Edot = matrix[0,:]

In [32]:
max_index = int( (max_deg+max_deg*(max_deg+1)/2)*2-4 )        # which includes sin terms with order m = 0
n = n[1:max_index+1]
m = m[1:max_index+1]
matrix = matrix[1:max_index+1,:]
tmp_index = np.linspace(0,np.size(m)-1,np.size(m),dtype=int)
tmp_flag = (tmp_index*0 + 1).astype(bool)       # default True
tmp_flag[  tmp_index[m==0][1::2]  ] = False
n = n[tmp_flag]
m = m[tmp_flag]
matrix = matrix[tmp_flag,:]

In [33]:
A = np.asmatrix( matrix ).T
b = np.asmatrix( slope - Edot ).T

In [34]:
b

matrix([[ 4.28176441e-03],
        [ 1.45499654e-04],
        [ 1.42270185e-02],
        [ 5.58347371e-05],
        [-1.90189661e-04],
        [ 6.06248927e-04],
        [ 1.02424016e-03],
        [ 1.17832345e-02],
        [ 8.24594792e-04],
        [ 1.74645723e-03],
        [ 1.99504274e-03],
        [ 4.13300545e-03],
        [-7.52820423e-04],
        [ 1.05060615e-02],
        [ 3.19016297e-03],
        [-4.79512687e-04],
        [ 8.78035083e-04],
        [ 6.37159132e-04],
        [ 2.85576787e-03],
        [ 4.15660182e-04],
        [-5.47354428e-04],
        [ 7.60248501e-03],
        [ 2.03504736e-03],
        [ 1.69954166e-03],
        [-1.58331571e-03],
        [ 8.65436362e-03],
        [ 6.90708742e-04],
        [-1.49838268e-03],
        [ 1.44658346e-03],
        [ 3.61411341e-03],
        [ 1.52483890e-03],
        [ 1.83824527e-03],
        [ 1.00574981e-03],
        [ 6.80918282e-03],
        [ 8.59412097e-03],
        [ 2.99656431e-03],
        [ 3.54853038e-03],
 

In [35]:
x = (A.T * A).I * A.T * b

In [36]:
x

matrix([[ 5.29980834e-11],
        [ 5.34740627e-11],
        [-9.99705775e-11],
        [ 5.34837484e-11],
        [-1.45875700e-09],
        [ 2.85478171e-11],
        [ 3.80002376e-11],
        [-1.53246439e-11],
        [-8.42071436e-11],
        [ 8.10641884e-11],
        [-1.25379588e-10],
        [-4.18155565e-10],
        [ 6.54280231e-12],
        [-1.74597685e-11],
        [ 3.12472704e-11],
        [-6.75919367e-12],
        [ 2.27389258e-10],
        [ 3.02442516e-11],
        [ 1.64319183e-10],
        [ 2.53738732e-11],
        [ 1.04516425e-10]])

In [37]:
estimates = np.asarray(x)
tmp_index = np.linspace(0,np.size(m)-1,np.size(m),dtype=int)
tmp_flag = (tmp_index*0).astype(bool)       # default False
tmp_flag[ m == 0 ] = True
insert_index = tmp_index[tmp_flag] + 1
n = np.insert(n,insert_index,n[tmp_flag])
m = np.insert(m,insert_index,m[tmp_flag])
estimates = np.insert(estimates,insert_index,0)

In [38]:
coeff1 = np.column_stack((n[0::2],m[0::2],estimates[0::2],estimates[1::2]))

In [39]:
station_list_data = np.genfromtxt('list.txt')

In [40]:
lat = station_list_data[:,1]
lon = station_list_data[:,2]

In [41]:
colat = ( 90 - lat )*np.pi/180.0
lon = ( lon - 360.0*(lon > 180.0).astype(float) )*np.pi/180.0

In [42]:
load_love_file_name = 'Load_Love2_CM.dat'
load_love_pd_dataframe =  read_csv(load_love_file_name, header = None, skiprows = 13, sep = '\s+',
        dtype = str)
load_love_pd_dataframe[1] = load_love_pd_dataframe[1].str.replace('D','E')
load_love_pd_dataframe[2] = load_love_pd_dataframe[2].str.replace('D','E')
load_love_pd_dataframe[3] = load_love_pd_dataframe[3].str.replace('D','E')
load_love_data = np.asarray(load_love_pd_dataframe,
        dtype = float)
h = load_love_data[:,1]
k = load_love_data[:,3]
    

In [43]:
radius_of_earth = 6378100.0     # in meters m
Fv = radius_of_earth*(1.1677 * load_love_data[:,0]  - 0.5233)     # Fv values depends on the degree, first column of load_love_data is degree
Fe =  radius_of_earth * h/(1+k) # This has inf and nan
Fe[0] = 0   # in CM frame degree 0 and 1 cannot be calculated
Fe[1] = 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
def nalf(n,m,x):
    # order degree input
    return lpmv(m,n,x) * (-1)**m * sqrt( (2-float(m==0)) * (2.0*n + 1) * fact(n-m)/fact(n+m) )

In [45]:
n = coeff1[:,0].astype(int)
m = coeff1[:,1].astype(int)
C = coeff1[:,2].astype(float)
S = coeff1[:,3].astype(float)

In [56]:
elastic1 = np.zeros(len(lat))
for j in range(len(lat)):
    for i in range( np.size(n)):
        elastic1[j] += nalf( n[i] , m[i] , cos(colat[j]) ) * ( Fv[n[i]] - Fe[n[i]] ) * ( C[i]*cos(m[i]*lon[j]) + S[i]*sin(m[i]*lon[j]) )

In [57]:
elastic1


array([ 5.09037670e-03,  2.49488772e-03,  6.92185112e-03, -1.01310073e-03,
        1.08102758e-03,  5.30160859e-04,  5.53180571e-05,  7.42015820e-03,
        1.66958478e-03,  6.63141157e-03,  2.69224727e-03,  5.94221140e-03,
        6.51740864e-04,  1.92633561e-03,  6.02823499e-03,  1.06639934e-03,
        1.06639907e-03,  1.06639901e-03,  9.31909783e-04,  9.31564851e-04,
        3.23259959e-03,  6.09682671e-03, -1.07642131e-03,  2.00347973e-03,
       -1.93729677e-03,  4.48685715e-03,  8.97815875e-04, -5.67244483e-04,
        3.22565869e-03,  4.94056395e-03,  1.44486522e-03,  3.96076979e-03,
        9.41263518e-04,  5.74502214e-03,  5.74501400e-03,  4.79772112e-03,
        4.79769815e-03,  3.03862807e-03,  2.33021483e-03, -6.34668851e-04,
       -4.29405718e-03, -4.29502067e-03,  1.12102501e-03,  7.26658659e-04,
        1.05762449e-03,  3.24433835e-03,  1.62669438e-03,  3.65303009e-03,
        8.01809225e-04,  8.94466238e-03,  2.86655132e-03,  3.22447450e-03,
        4.01822697e-05,  

In [58]:
b1 = np.asmatrix(np.asarray(b)[:,0] - elastic1).T
#b1 = np.asmatrix( elastic1 - Edot ).T


In [59]:
b1[4] , b[4] , elastic1[4]

(matrix([[-0.00127122]]), matrix([[-0.00019019]]), 0.0010810275832615899)

In [60]:
(A.T * A).I * A.T * b1

matrix([[-4.30957868e-25],
        [ 1.08890573e-23],
        [ 4.30553971e-24],
        [-2.03434732e-23],
        [-7.96419834e-23],
        [-4.76598204e-25],
        [-9.79449700e-26],
        [-1.00974196e-24],
        [ 2.30867401e-24],
        [ 7.10858339e-25],
        [ 2.29801114e-23],
        [ 6.69499308e-24],
        [ 8.99882034e-25],
        [-3.58821902e-24],
        [-8.56261181e-25],
        [ 1.75129645e-24],
        [ 1.31928845e-23],
        [-3.87094677e-24],
        [-3.10192730e-25],
        [-1.40943822e-23],
        [-1.20748982e-23]])

In [55]:
x


matrix([[ 5.29980834e-11],
        [ 5.34740627e-11],
        [-9.99705775e-11],
        [ 5.34837484e-11],
        [-1.45875700e-09],
        [ 2.85478171e-11],
        [ 3.80002376e-11],
        [-1.53246439e-11],
        [-8.42071436e-11],
        [ 8.10641884e-11],
        [-1.25379588e-10],
        [-4.18155565e-10],
        [ 6.54280231e-12],
        [-1.74597685e-11],
        [ 3.12472704e-11],
        [-6.75919367e-12],
        [ 2.27389258e-10],
        [ 3.02442516e-11],
        [ 1.64319183e-10],
        [ 2.53738732e-11],
        [ 1.04516425e-10]])